In [2]:
import os
import csv
import cv2
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

# CSV 파일 읽기
df = pd.read_csv('한국농수산식품유통공사_전통주 정보_20230301.csv', encoding='cp949')
print(df.iloc[:, 0])
search_list = df.iloc[:, 0].tolist()  # 첫 10개 검색어만 사용

# Chrome WebDriver를 시작합니다.
driver = webdriver.Chrome()
driver.get('https://www.sooldamhwa.com')

# 술 리스트
liquor_list = []

# 술 정보 검색 함수
def search(name):
    
    time.sleep(3)
    # 홈페이지로 이동
    wait = WebDriverWait(driver, 30)
    homepage = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[1]/div/section[1]/ul/li[3]/div/span')))
    homepage.click()
    
    # 검색창 클릭
    time.sleep(1)
    input_box = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div[1]/div/form/input')
    input_box.clear()
    input_box.send_keys(name)
    input_box.send_keys(Keys.RETURN)
    
    found_drinks = []
    try:
        element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div/div/div/div[2]/div/div[1]/div/a'))
        WebDriverWait(driver, 3).until(element_present)
        
        for idx in range(1, 31):  # 시작 인덱스를 1로 변경 (1부터 30까지 검사)
            # 예상 검색 결과 요소 선택
            result_element = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div/div/div/div[2]/div/div[{}]/div/a'.format(idx))
            
            if "선물" in result_element.text:
                print(f"{name} 선물세트는 배제됩니다.")
                print("=" * 50)
                continue  # 다음 술을 검색합니다.
            else:
                # 검색 결과 링크 클릭
                result_element.click()
                found_drinks.append(name)
                liquor_list.append(name)
                # print(f"찾은 술: {name}")
                return idx, found_drinks

    except TimeoutException:
        print(f"검색 결과가 없습니다: {name}")
        print("=" * 50)
        return None, []
    except NoSuchElementException:
        pass  # 해당 요소를 찾지 못해도 넘어갑니다.
    
    # 검색 결과가 없는 경우 빈 리스트 반환
    if not found_drinks:
        return None, []
    return idx, found_drinks, result_element

# 주종
def type_liquor():
    try:
        type_liquor_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[3]/div[1]/span'))
        )
        type_liquor_text = type_liquor_element.text
        return type_liquor_text
    except TimeoutException:
        return "준비 중입니다."

# 도수
def number_alcohol():
    try:
        number_alcohol_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[3]/div[2]/span'))
        )
        number_liquor_text = number_alcohol_element.text
        return number_liquor_text
    except TimeoutException:
        return "준비 중입니다."

# 유통기한
def expiration_date():
    try:
        expiration_date_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[4]/div[3]/span'))
        )
        expiration_date_text = expiration_date_element.text
        return expiration_date_text
    except TimeoutException:
        return "준비 중입니다."

# 보관방법
def storage_method():
    try:
        storage_method_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[4]/div[4]/span'))
        )
        storage_method_text = storage_method_element.text
        return storage_method_text
    except TimeoutException:
        return "준비 중입니다."

# 어울리는 안주
def all_side_dishes():
    try:
        # 부모 XPATH를 사용하여 모든 어울리는 안주들을 찾습니다.
        side_dishes_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[1]/div[4]/div[2]/div[*]/label'))
        )
    
        # 각 안주의 텍스트를 리스트에 저장합니다.
        side_dishes_text = [dish.text for dish in side_dishes_elements]
        side_dishes_texts = ", ".join(side_dishes_text)
        

        return side_dishes_texts
    
    except TimeoutException:
        return "준비 중입니다."


image_score_dict = {
    "/images/modules/damhwaMarket/detail/icon_rating_zero.png" : 0,
    "/images/modules/damhwaMarket/detail/icon_rating_one.png": 1,
    "/images/modules/damhwaMarket/detail/icon_rating_two.png": 2,
    "/images/modules/damhwaMarket/detail/icon_rating_three.png": 3,
    "/images/modules/damhwaMarket/detail/icon_rating_four.png": 4,
    "/images/modules/damhwaMarket/detail/icon_rating_five.png": 5,
}

def get_taste_information():
    try:
        # 현재 페이지의 HTML 내용을 가져옵니다.
        html_code = driver.page_source
        # BeautifulSoup를 사용하여 HTML을 파싱합니다.
        soup = BeautifulSoup(html_code, 'html.parser')

        # 맛 점수를 저장할 빈 딕셔너리를 생성합니다.
        taste_score = {}

        # CSS 선택자를 사용하여 맛 점수 이미지에 해당하는 img 태그들과 맛 태그들을 가져옵니다.
        img_tags = soup.select('div.taste-rating-bar > div > div > span > img')
        name_tags = soup.select('div.SubscriptionHintRatingStyles__RatingName-sc-8yfc2n-1.cobuTZ')

        # 이미지 태그와 맛 태그를 짝지어 맛 점수를 추출합니다.
        for img_tag, name_tag in zip(img_tags, name_tags):
            if img_tag.get('alt') == "향의 강도 그래프 바 이미지":
                # 이미지 URL을 키로 사용하고, 이미지에 해당하는 맛 점수를 딕셔너리에 추가합니다.
                taste_score[name_tag.text.strip()] = image_score_dict[img_tag['src']]

        return taste_score

    except TimeoutException:
        return None




# 검색어 리스트를 순회하며 술 정보를 검색하고 DataFrame에 추가합니다.
for drink_name in search_list:
    idx, found_drinks = search(drink_name)
    
    # 각 술의 정보를 리스트로 저장합니다.
    liquor_info = [drink_name, "", "", "", "", "", "", "", ""]
    
    if found_drinks:
        print(f"술: {drink_name}")
        
        # 주종, 도수, 유통기한, 보관방법, 어울리는 안주 정보를 갱신합니다.
        type_liquor_result = type_liquor()
        number_alcohol_result = number_alcohol()
        expiration_date_result = expiration_date()
        storage_method_result = storage_method()
        taste_score_result = get_taste_information()
        all_side_dishes_result = all_side_dishes()
        print(f"주종: {type_liquor_result}")
        print(f"도수: {number_alcohol_result}")
        print(f"유통기한: {expiration_date_result}")
        print(f"보관방법: {storage_method_result}")
        print(f"어울리는 안주: {all_side_dishes_result}")
        print(f"맛 점수: {taste_score_result}")
        print("=" * 50)
        
        # 술 정보를 리스트에 추가합니다.
        liquor_info[1] = type_liquor_result
        liquor_info[2] = number_alcohol_result
        liquor_info[3] = expiration_date_result
        liquor_info[4] = storage_method_result
        liquor_info[6] = all_side_dishes_result  # 어울리는 안주들을 문자열로 합쳐서 저장
        liquor_info[8] = taste_score_result

        # 데이터 입력 및 저장
        with open('TeamProject.csv', 'a', newline='') as csvfile:
            ProjectDataFrame = csv.writer(csvfile)
            ProjectDataFrame.writerow(liquor_info)

print("Found drinks:")
print(f"총 {len(liquor_list)}개 입니다.")
print(liquor_list)



0             입장탁주
1          톡 한잔 소주
2      자연담은 복분자막걸리
3          로얄 안동소주
4             지란지교
          ...     
846             화백
847         산천어막걸리
848    황금보리 증류주 17
849    황금보리 증류주 25
850    황금보리 증류주 40
Name: 전통주명, Length: 851, dtype: object
검색 결과가 없습니다: 입장탁주
검색 결과가 없습니다: 톡 한잔 소주
술: 자연담은 복분자막걸리
주종: 살균탁주
도수: 6.00%
유통기한: 제조일로부터 1년
보관방법: 냉장 보관
어울리는 안주: 전복 버터구이, 삼계탕, 장어구이
맛 점수: {'단맛': 4, '바디': 2, '산미': 2, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 로얄 안동소주
지란지교 선물세트는 배제됩니다.
지란지교 선물세트는 배제됩니다.
술: 지란지교
주종: 탁주
도수: 12.00%
유통기한: 병입일로부터 90일까지
보관방법: 냉장 보관
어울리는 안주: 불족발, 곱창, 훈제오리 냉채
맛 점수: {'단맛': 4, '바디': 4, '산미': 3, '탄산': 0, '씁쓸': 2}
술: 지란지교 무화과 탁주
주종: 탁주
도수: 12.00%
유통기한: 병입일로부터 90일까지
보관방법: 냉장 보관
어울리는 안주: 불족발, 곱창, 훈제오리 냉채
맛 점수: {'단맛': 4, '바디': 4, '산미': 3, '탄산': 0, '씁쓸': 2}
술: 지란지교 약주
주종: 약주
도수: 15.00%
유통기한: 제조일로부터 180일까지
보관방법: 냉장 보관
어울리는 안주: 간장 치킨, 굴국, 버섯 볶음
맛 점수: {'단맛': 3, '씁쓸': 2, '산미': 4, '바디': 3}
술: 꽃 와인
주종: 과실주
도수: 11.50%
유통기한: 유통기한 없음
보관방법: 직사광선은 피하고 서늘한 곳에 보관
어울리는 안주: 준비 중입니다.
맛 점수: {'단맛': 3, '바디': 

검색 결과가 없습니다: 금정산성막걸리
술: 금풍양조
주종: 탁주
도수: 6.90%
유통기한: 병입일로부터 30일
보관방법: 냉장 보관
어울리는 안주: 메밀 전병, 도토리묵 무침, 두부김치
맛 점수: {'단맛': 3, '바디': 3, '산미': 2, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 과하주23
검색 결과가 없습니다: 김천대학교 자두사랑 자두와인
검색 결과가 없습니다: 미생 막걸리
술: 선호 생막걸리
주종: 탁주
도수: 6.00%
유통기한: 병입일로부터 20일
보관방법: 냉장 보관
어울리는 안주: 두부구이, 나쵸, 해물 파전
맛 점수: {'단맛': 2, '바디': 1, '산미': 3, '탄산': 4, '씁쓸': 2}
술: 김포특주
주종: 살균약주
도수: 15%
유통기한: 제조일로부터 2년
보관방법: 직사광선을 피하고 서늘한 곳에 보관
어울리는 안주: 간장 게장, 감자탕, 시금치 무침
맛 점수: {'단맛': 3, '씁쓸': 4, '산미': 2, '바디': 3}
검색 결과가 없습니다: 한강의설레임
술: 김포 막걸리
주종: 탁주
도수: 6.00%
유통기한: 병입일로부터 20일
보관방법: 냉장 보관
어울리는 안주: 두부구이, 나쵸, 해물 파전
맛 점수: {'단맛': 2, '바디': 1, '산미': 3, '탄산': 4, '씁쓸': 2}
검색 결과가 없습니다: 김포예주
검색 결과가 없습니다: 김포별주
검색 결과가 없습니다: 김포약주
검색 결과가 없습니다: 김포예주 프리미엄
검색 결과가 없습니다: 까메오 막걸리
검색 결과가 없습니다: 지장수 호박 생 막걸리
검색 결과가 없습니다: 지장수 호박막걸리
검색 결과가 없습니다: 낙천 옥수수 생동동주
검색 결과가 없습니다: 열대야
검색 결과가 없습니다: 정고집 나주배 약주 9%
검색 결과가 없습니다: 정고집 나주배 약주 15%
검색 결과가 없습니다: 남한산성소주
검색 결과가 없습니다: 남한산성소주 21
검색 결과가 없습니다: 강소주
술: 화주
주종: 증류식소주
도수: 40.00%
유통기한: 유통기한 없음
보관방법:

검색 결과가 없습니다: 화전일취 15
술: 선비 진
주종: 일반증류주
도수: 48%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 과일 샐러드, 카프레제, 문어 숙회
맛 점수: {'여운': 5, '바디': 4}
검색 결과가 없습니다: 선비 보드카
술: 토끼 소주
주종: 증류식소주
도수: 40%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 크림 새우, 양꼬치, 꿔바로우
맛 점수: {'여운': 4, '바디': 4}
냥이탁주 화이트 선물세트는 배제됩니다.
술: 냥이탁주 화이트
주종: 탁주
도수: 10.5%
유통기한: 제조일로부터 60일
보관방법: 냉장 보관
어울리는 안주: 메밀소바, 삼치구이, 백순대 볶음
맛 점수: {'단맛': 3, '바디': 3, '산미': 3, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 지리산 강쇠
검색 결과가 없습니다: 주몽복분자주
검색 결과가 없습니다: 황진이주
검색 결과가 없습니다: 천마이야기 막걸리
검색 결과가 없습니다: 블루베리 이야기
검색 결과가 없습니다: 오마이갓스파클링
송도막걸리 선물세트는 배제됩니다.
검색 결과가 없습니다: 도원결의
검색 결과가 없습니다: 해방주
술: 대대포 막걸리
주종: 탁주
도수: 6.00%
유통기한: 병입일로부터 30일
보관방법: 냉장 보관
어울리는 안주: 양념 치킨, 불닭볶음면, 떡볶이
맛 점수: {'단맛': 3, '바디': 2, '산미': 2, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 도깨비술 11
검색 결과가 없습니다: 애석
술: 진맥소주 22%
주종: 증류식소주
도수: 22.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 떡갈비, 수육, 전어회
맛 점수: {'여운': 2, '바디': 3}
술: 볼빨간막걸리 10
주종: 탁주
도수: 10.00%
유통기한: 병입일로부터 2개월
보관방법: 냉장 보관
어울리는 안주: 맥앤치즈, 돼지국밥, 닭곰탕
맛 점수: {'단맛': 2, '바디': 4, '산미': 4, '탄산

술: DOK막걸리
주종: 탁주
도수: 6.0%
유통기한: 제조일로부터 30일
보관방법: 냉장 보관
어울리는 안주: 닭발, 아귀찜, 주꾸미 볶음
맛 점수: {'단맛': 4, '바디': 3, '산미': 2, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 레드듀 RED DEW
술: 얼떨결에
주종: 탁주
도수: 6.00%
유통기한: 제조일로부터 40일
보관방법: 냉장 보관
어울리는 안주: 까르보나라, 맥앤치즈, 피자
맛 점수: {'단맛': 3, '바디': 2, '산미': 4, '탄산': 4, '씁쓸': 1}
술: 얼떨결에 퍼플
주종: 탁주
도수: 6.00%
유통기한: 제조일로부터 40일
보관방법: 냉장 보관
어울리는 안주: 까르보나라, 맥앤치즈, 피자
맛 점수: {'단맛': 3, '바디': 2, '산미': 4, '탄산': 4, '씁쓸': 1}
검색 결과가 없습니다: 부안참뽕막걸리
부안참뽕와인 선물세트는 배제됩니다.
술: 부안참뽕와인
주종: 과실주
도수: 13.00%
유통기한: 유통기한 없음
보관방법: 직사광선을 피하고 서늘한 곳에 보관
어울리는 안주: 블루베리 치즈 케이크, 장어구이, 떡볶이
맛 점수: {'단맛': 4, '바디': 3, '산미': 2, '탄산': 0, '탄닌': 2}
검색 결과가 없습니다: 영일만친구 막걸리
검색 결과가 없습니다: 도구막걸리
검색 결과가 없습니다: 세오녀탁주
검색 결과가 없습니다: 동해동동주
검색 결과가 없습니다: 연오랑탁주
검색 결과가 없습니다: 북극곰 막걸리
검색 결과가 없습니다: 도구막걸리
술: 메밀로
주종: 일반증류주
도수: 25.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 막국수, 물회, 미나리 삼겹살
맛 점수: {'여운': 4, '바디': 4}
검색 결과가 없습니다: 대관람차
검색 결과가 없습니다: 술헤는밤
검색 결과가 없습니다: 희양산 막걸리 15도
술: 희양산 막걸리 9도
주종: 탁주
도수: 9.00%
유통기한: 병입일로부터 30일
보관방법: 냉장 보관
어울리는 안주: 탕수육, 칠리

검색 결과가 없습니다: 복분자아락40
검색 결과가 없습니다: 오디아락40
검색 결과가 없습니다: 느린마을 증류주
검색 결과가 없습니다: 심술 알쓰
검색 결과가 없습니다: 느린마을 약주
검색 결과가 없습니다: 우곡생주
검색 결과가 없습니다: 호두땅콩막걸리
검색 결과가 없습니다: 메로니아
검색 결과가 없습니다: 호랑이 생 막걸리
검색 결과가 없습니다: 메로니아
검색 결과가 없습니다: 매실탁테일
술: 이담
주종: 일반증류주
도수: 33.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 딱새우 회, 전어회, 갈비 구이
맛 점수: {'여운': 4, '바디': 3}
검색 결과가 없습니다: 로아40 옐로우
검색 결과가 없습니다: 우곡주
검색 결과가 없습니다: 매화마름 호랑이 막걸리
검색 결과가 없습니다: 부자 송산포도
술: 부자
주종: 일반증류주
도수: 44.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 육포, 목살구이, 참치타다끼
맛 점수: {'여운': 4, '바디': 4}
검색 결과가 없습니다: 배도가 로아 40도 옐로우
검색 결과가 없습니다: 로아40 레드
검색 결과가 없습니다: 배도가 심플리 애플
검색 결과가 없습니다: 매화마름
검색 결과가 없습니다: 로아19레드
검색 결과가 없습니다: 로아19옐로우
검색 결과가 없습니다: 로아40퍼플
검색 결과가 없습니다: 로아19화이트
검색 결과가 없습니다: 로아40화이트
검색 결과가 없습니다: 생 옥수수 동동주
검색 결과가 없습니다: 매실 막걸리
검색 결과가 없습니다: 백운 복분자 와인
검색 결과가 없습니다: 백운복분자주
술: 매화깊은밤
주종: 살균약주
도수: 8.00%
유통기한: 제조일로부터 2년
보관방법: 상온 보관 가능
어울리는 안주: 차돌박이 숙주 볶음, 시저 샐러드, 백순대 볶음
맛 점수: {'단맛': 4, '씁쓸': 3, '산미': 3, '바디': 2}
술: 보리수헤는밤
주종: 살균약주
도수: 8.00%
유통기한: 제조일로부터 2년
보관방법: 상온 보관 가능
어울

...

검색 결과가 없습니다: PS 애플시드르
술: 미상25
주종: 일반증류주
도수: 25.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 삼겹살, 문어 숙회, 해삼
맛 점수: {'여운': 3, '바디': 3}
검색 결과가 없습니다: 귀감
신례명주 선물세트는 배제됩니다.
술: 말이야 막걸리야
주종: 탁주
도수: 6%
유통기한: 제조일로부터 45일
보관방법: 냉장 보관
어울리는 안주: 소시지, 어묵전골, 라면
맛 점수: {'단맛': 2, '바디': 2, '산미': 1, '탄산': 2, '씁쓸': 1}
술: 시향가 미니캔막걸리
주종: 3일 이내 배송
도수: 준비 중입니다.
유통기한: 준비 중입니다.
보관방법: 준비 중입니다.
어울리는 안주: 준비 중입니다.
맛 점수: {}
검색 결과가 없습니다: 산막생대덕주
술: 백련 맑은술
주종: 약주
도수: 12.00%
유통기한: 제조일로부터 90일
보관방법: 냉장 보관
어울리는 안주: 떡갈비, 된장찌개, 누룽지탕
맛 점수: {'단맛': 2, '씁쓸': 3, '산미': 4, '바디': 3}
검색 결과가 없습니다: 백련 생막걸리 snow (스노우)
검색 결과가 없습니다: 하얀연꽃 백련생막걸리
술: 하얀연꽃 백련막걸리
주종: 살균탁주
도수: 7.00%
유통기한: 제조일로부터 1년
보관방법: 상온 보관 가능
어울리는 안주: 순대 볶음, 감자전, 갈비찜
맛 점수: {'단맛': 3, '바디': 3, '산미': 2, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 하얀연꽃 백련맑은 술
검색 결과가 없습니다: 씨(C) 그린막걸리
검색 결과가 없습니다: 씨(C) 옐로막걸리
검색 결과가 없습니다: 씨(C) 막걸리 시그니처 큐베
검색 결과가 없습니다: 술공방 9.0 막걸리
술: 허니문
주종: 기타주류
도수: 10.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 딸기, 약과, 마카롱
맛 점수: {'단맛': 3, '바디': 2, '산미': 2, '탄산': 0, '탄닌': 1}
술: 허니비
주종:

술: 동몽
주종: 약주
도수: 17.00%
유통기한: 병입일로부터 60일
보관방법: 냉장 보관
어울리는 안주: 연어 스테이크, 오리 구이, 두부부침
맛 점수: {'단맛': 3, '씁쓸': 3, '산미': 4, '바디': 4}
술: 배꽃 필 무렵
주종: 탁주
도수: 9.00%
유통기한: 병입일로부터 1개월까지
보관방법: 10˚C 이하 냉장 보관
어울리는 안주: 마카롱, 스테이크, 초콜릿
맛 점수: {'단맛': 3, '바디': 5, '산미': 4, '탄산': 0, '씁쓸': 2}
검색 결과가 없습니다: 홍천강 탁주
검색 결과가 없습니다: 남산애 포트와인
검색 결과가 없습니다: 남산애 레드와인
검색 결과가 없습니다: 영탁막걸리
검색 결과가 없습니다: 오계리 아이스와인
검색 결과가 없습니다: 감자술 13%
검색 결과가 없습니다: 서주 16%
검색 결과가 없습니다: 다래와인 7004S
검색 결과가 없습니다: 다래와인 7004D
검색 결과가 없습니다: 다래와인 3004
술: 문경바람 오크
주종: 일반증류주
도수: 40.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 육포, 견과류, 초콜릿
맛 점수: {'여운': 4, '바디': 4}
고운달 백자 선물세트는 배제됩니다.
고운달 오크 선물세트는 배제됩니다.
술: 오미로제 연
주종: 과실주
도수: 8.00%
유통기한: 유통기한 없음
보관방법: 직사광선을 피하고 서늘한 곳에 눕혀서 보관
어울리는 안주: 새우 오일 파스타, 연어샐러드, 육전
맛 점수: {'단맛': 1, '바디': 3, '산미': 4, '탄산': 3, '탄닌': 1}
스파클링와인 오미로제 결 선물세트는 배제됩니다.
오미로제 프리미어와인 선물세트는 배제됩니다.
오미로제 프리미어와인 선물세트는 배제됩니다.
술: 오미로제 프리미어와인
주종: 과실주
도수: 12.00%
유통기한: 유통기한 없음
보관방법: 직사광선을 피하고 서늘한 곳 보관
어울리는 안주: 버섯구이, 소고기 등심구이, 더덕구이
맛 점수: {'단맛': 2, '바디': 2, '산미': 

술: 이강주
주종: 리큐르
도수: 25.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 떡갈비, 육전, 데리야끼 치킨
맛 점수: {'단맛': 2, '여운': 3, '바디': 3}
검색 결과가 없습니다: 경기전
술: 오디여라
주종: 일반증류주
도수: 25.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 크림소스 연어 스테이크, 양갈비 구이, 콘치즈
맛 점수: {'여운': 3, '바디': 3}
검색 결과가 없습니다: 청진주
술: 아리아리
주종: 일반증류주
도수: 40.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 과일 샐러드, 월남쌈, 냉채 족발
맛 점수: {'여운': 3, '바디': 3}
검색 결과가 없습니다: 곤드레생막걸리
검색 결과가 없습니다: 청혼골드
검색 결과가 없습니다: 청혼 화이트
검색 결과가 없습니다: 청혼 레드25
검색 결과가 없습니다: 청혼 블루
술: 문경바람 백자 25
주종: 일반증류주
도수: 25.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 과일 샐러드, 리코타치즈, 견과류
맛 점수: {'여운': 3, '바디': 2}
술: 문경바람 백자 40
주종: 일반증류주
도수: 40.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 삼겹살, 곱창, 고등어 회
맛 점수: {'여운': 3, '바디': 3}
검색 결과가 없습니다: 기다림 34
검색 결과가 없습니다: 기다림 25
검색 결과가 없습니다: 기다림 16
검색 결과가 없습니다: 기다림 맑은술
술: 동래아들
주종: 탁주
도수: 6.00%
유통기한: 병입일로부터 45일
보관방법: 냉장 보관
어울리는 안주: 닭볶음탕, 아귀찜, 떡볶이
맛 점수: {'단맛': 4, '바디': 3, '산미': 1, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 제주술익는집 고소리술
술: 오메기 맑은술
주종: 약주
도수: 16%
유통기한: 병입일로부터 6개월
보관방법: 냉장 보관
어울

검색 결과가 없습니다: 부자진 오미자 진
검색 결과가 없습니다: 부자진 배치#0001
검색 결과가 없습니다: 사임주
검색 결과가 없습니다: 삼해소주
검색 결과가 없습니다: 선운산 복분자주 프리미엄
검색 결과가 없습니다: 선운산 복분자주
검색 결과가 없습니다: 선운산 복분자와인
검색 결과가 없습니다: 샹스프랑보아즈
술: 미담
주종: 탁주
도수: 10%
유통기한: 제조일로부터 60일
보관방법: 냉장 보관
어울리는 안주: 밀푀유나베, 차돌박이 숙주 볶음, 팟타이
맛 점수: {'단맛': 2, '바디': 3, '산미': 2, '탄산': 0, '씁쓸': 2}
검색 결과가 없습니다: 수성 프리미엄 백주43
술: 혼디주
주종: 과실주
도수: 12.00%
유통기한: 유통기한 없음
보관방법: 직사광선을 피하고 서늘한 곳에 보관
어울리는 안주: 과일 샐러드, 밀푀유나베, 생선회
맛 점수: {'단맛': 3, '바디': 3, '산미': 4, '탄산': 0, '탄닌': 0}
술: 한청
주종: 청주
도수: 15.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 비빔면, 유부초밥, 동치미막국수
맛 점수: {'단맛': 3, '씁쓸': 2, '산미': 3, '바디': 3}
검색 결과가 없습니다: 고향생막걸리
검색 결과가 없습니다: 고향인삼막걸리
검색 결과가 없습니다: SSOA-포도
술: 고리진
주종: 일반증류주
도수: 47.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 조개찜, 견과류, 크래커
맛 점수: {'여운': 4, '바디': 4}
검색 결과가 없습니다: 금과 명주
검색 결과가 없습니다: 애피소드호프
검색 결과가 없습니다: The 찾을수록
술: 애피소드 상그리아
주종: 과실주
도수: 3.50%
유통기한: 유통기한 없음
보관방법: 직사광선을 피해 서늘한 곳에 보관 또는 냉장보관
어울리는 안주: 과일치즈, 크래커, 쉬폰 케이크
맛 점수: {'단맛': 3, '바디': 3, '산미': 3, '탄산': 2, '탄닌': 2}
술: 애피소드 애플

검색 결과가 없습니다: 산머루와인 달콤한 스위트
검색 결과가 없습니다: 산머루와인 깊은맛 드라이
술: 오크와인
주종: 과실주
도수: 10.50%
유통기한: 유통기한 없음
보관방법: 직사광선을 피하고 서늘한 곳에 세워서 보관
어울리는 안주: 과일 샐러드, 고르곤졸라 피자, 흰살생선회
맛 점수: {'단맛': 4, '바디': 2, '산미': 3, '탄산': 0, '탄닌': 0}
검색 결과가 없습니다: 2018년산 하미앙 오크와인
검색 결과가 없습니다: 2016년산 하미앙 드라이
검색 결과가 없습니다: 2016년산 하미앙 스위트
검색 결과가 없습니다: 하향주
검색 결과가 없습니다: 나루 생 막걸리 6도
검색 결과가 없습니다: 나루 생 막걸리 11.5도
검색 결과가 없습니다: 김수한무
검색 결과가 없습니다: 계룡산 산삼주 금수강산 21%
검색 결과가 없습니다: 계룡산 산삼주 금수강산 40%
술: 헤베
주종: 과실주
도수: 9.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 막국수, 고르곤졸라 피자, 초밥
맛 점수: {'단맛': 2, '바디': 2, '산미': 4, '탄산': 2, '탄닌': 2}
검색 결과가 없습니다: 스타베리오디
검색 결과가 없습니다: 뱅꼬레 아이스와인
검색 결과가 없습니다: 뱅꼬레 로제와인
검색 결과가 없습니다: 뱅꼬레 레드와인
검색 결과가 없습니다: 뱅꼬레 화이트와인
검색 결과가 없습니다: 뱅꼬레 더감
술: 불소곡주
주종: 일반증류주
도수: 41.00%
유통기한: 유통기한 없음
보관방법: 상온 보관 가능
어울리는 안주: 조개구이, 산낙지, 불고기
맛 점수: {'여운': 3, '바디': 3}
술: 한산소곡주
주종: 약주
도수: 16.00%
유통기한: 병입일로부터 6개월
보관방법: 냉장 보관
어울리는 안주: 샤브샤브, 제육 볶음, 약과
맛 점수: {'단맛': 4, '씁쓸': 3, '산미': 2, '바디': 4}
검색 결과가 없습니다: 일오백화이트
검색 결과가 없습니다: 일오백골드
술: 불소주
주종: 증류식소주
도수: 25.0

In [3]:
import os
import cv2

def compare_image(image1_file, image2_path):
    image1 = cv2.imread(image1_file)
    image2 = cv2.imread(image2_path)
    
    if image1 is None or image2 is None:
        raise ValueError("Failed to read one or both images. Please check the file paths.")
    
    # 이미지 크기를 맞추기 위해 resize
    image2_resized = cv2.resize(image2, (image1.shape[1], image1.shape[0]))
    
    # 이미지를 그레이스케일로 변환
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(image2_resized, cv2.COLOR_BGR2GRAY)

    difference = cv2.subtract(image1_gray, image2_gray)
    non_zero_pixels = cv2.countNonZero(difference)
    
    if non_zero_pixels == 0:
        result = "두 사진은 동일합니다."
    else:
        result = "두 사진은 다른 사진입니다."
    
    return result

image_dict = {
       0: os.path.join("C:\TeamProject\image", "icon_rating_zero.png") ,
       1: os.path.join("C:\TeamProject\image", "icon_rating_one.png")  ,
       2: os.path.join("C:\TeamProject\image", "icon_rating_two.png")  ,
       3: os.path.join("C:\TeamProject\image", "icon_rating_three.png"),
       4: os.path.join("C:\TeamProject\image", "icon_rating_four.png") ,
       5: os.path.join("C:\TeamProject\image", "icon_rating_five.png"),
}

print(compare_image(image_dict[0], image_dict[6]))


KeyError: 6